In [31]:
import numpy as np
import pandas as pd


In [44]:
df = full_clean()

In [47]:
df[df['Use_Case_ID']== 'HHS-0076-2023']['Summary'].to_list()

['The goal of this project is to assess the potential of data��driven statistical methods for detecting and reducing coding differences between healthcare systems in Sentinel. Findings will inform development and deployment of methods and computational tools for transferring knowledge learned from one site to another and pave the way towards scalable and automated harmonization of electronic health records data.']

In [20]:
df['tech_edit'].value_counts()

tech_edit
Unspecified Machine Learning    247
All Other Techniques            151
Natural Language Processing     118
Neural Network                   85
Machine Vision                   55
Other                            54
Name: count, dtype: int64

In [12]:
df['tech_edit'].value_counts()

tech_edit
Unspecified Machine Learning    247
Other                           174
Natural Language Processing     118
Neural Network                   85
Machine Vision                   55
Automation                       31
Name: count, dtype: int64

In [9]:
test[['Title', 'Summary', 'Techniques', 'tech_edit','tech_clean']]

,Title,Summary,Techniques,tech_edit,tech_clean
0,Land Use Plan Document and Data Mining and Ana...,"Exploring the potential to identify patterns, ...",Natural Language Processing and Geo Classifica...,Natural Language Processing,Natural Language Processing
1,Seasonal/Temporary Wetland/Floodplain Delineat...,Reclamation was interested in determining if r...,Image classification using Joint Unsupervised ...,Other,Classification
2,Data Driven Sub-Seasonal Forecasting of Temper...,"Reclamation has run 2, year-long prize competi...","Range of data driven, AI/ML techniques (e.g. r...",Other,Classification
3,Data Driven Streamflow Forecasting,"Reclamation, along with partners from the CEAT...","Range of data driven, AI/ML techniques (e.g. L...",Neural Network,Neural Network
4,Snowcast Showdown,Reclamation partnered with Bonneville Power Ad...,"Range of data driven, AI/ML techniques",Unspecified Machine Learning,Unspecified Machine Learning
...,...,...,...,...,...
705,Extraction of family medical history from pati...,This pilot project uses TIU documentation on A...,NaN,Other,None
706,VA /IRB approved research study for finding co...,This IRB approved research study uses a rando...,NaN,Other,None
707,Interpretation/triage of eye images,Artificial intelligence supports triage of eye...,NaN,Machine Vision,Machine Vision
708,Screening for esophageal adenocarcinoma,National VHA administrative data is used to ad...,NaN,Other,None


In [104]:
test.to_csv('test.csv', index = False)

In [3]:
data = pd.read_csv("2023_ai_inventory_edit.csv")

In [71]:
no_tech = data[data['Techniques'].isna()]
no_tech['tech_trySum'] = None

/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_93319/1481991836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_tech['tech_trySum'] = None


In [ ]:
for d in other:
    print(f"""<option value="{d}">
            {d} ({len(data[data['Department'] == d])})
          </option>""")

In [24]:
top = df['tech_edit'].unique().tolist()

other = [d for d in df['tech_clean'].unique().tolist() if d not in top]

for d in other:
    print(f"""<option value="{d}">
            {d} ({len(df[df['tech_clean'] == d])})
          </option>""")

<option value="Classification">
            Classification (26)
          </option>
<option value="Regression">
            Regression (16)
          </option>
<option value="Other">
            Other (54)
          </option>
<option value="None">
            None (0)
          </option>
<option value="Large Language Model">
            Large Language Model (1)
          </option>
<option value="Automation">
            Automation (31)
          </option>
<option value="Time Series Forecast">
            Time Series Forecast (6)
          </option>


In [40]:
def full_clean(export:bool = True):
    data = pd.read_csv("2023_ai_inventory.csv")
    df_clean = process_data(data)

    for col in ['dev_edit', 'tech_edit', 'dep_edit']:
        df_clean = starting_grid(df_clean, col)
        df_clean = split_grid(df_clean, col)

    if export:
        df_clean.to_csv('2023_ai_inventory_edit.csv', index=False)

    return df_clean


In [43]:
def process_data(df):

    #clean up agency and Office
    df['Agency'] = df['Agency'].fillna('No Agency Provided')
    df['Office'] = df['Office'].fillna('No Office Provided')

    #clean up summary
    df['Summary'] = df['Summary'].str.replace("_x000D_", " ")
    # clean up dev stage
    # In Use
    stages = {'In Use': ['Operation and Maintenance','In-use', 'Implementation', 
                     'Operation and Management','In mission', 'In production: more than 1 year', 
                     'In production: less than six months', 'Completed','In production: less than one year', 'Deployment', 
                     'Technical transfer of capability to industry planned this summer', 'In production: less than 6 months' ,
                      'Technical transfer of capability to industry planned this summer.' , 'OPC runs in a pseudo-operational capacity via a webpage maintained by the Massachusetts Institute of Technology - Lincoln Lab, as well as in a test and evaluation capacity in a research mode.'],
          'In Planning': ['Initiation', 'Planned (not in production)', 'Investigating/Proof of concept' ],
          'In Development': ['Development and Acquisition', 'Development (not in production)', 'Proof-of-concept completed',
                             'Currently under development', "This is a pilot initiative", "User Acceptance Testing to begin early spring '22",
                               'Proof of Concept completed and published', 'Initial development', 'Refinments planned for future release', 'Successfully tested but not in production.'],
          'No Development Stage Indicated': [np.nan]
                }

    # Initialize the new column with None or any default value
    df['dev_edit'] = None

    # Iterate through the dictionary and update the new column
    for key, stage_list in stages.items():
        df.loc[df['Development_Stage'].isin(stage_list), 'dev_edit'] = key

    #clean up technique
    nlp = ['nlp', 'natural language']
    nn = ['neural network', 'nn', 'lstm', 'gans', 'synthetic']
    gml = ['machine learning', 'ml', 'ai', 'artificial intelligence', 
        'reinforcement','doodler', 'transfer']
    llm = ['large language']
    mv = ['machine vision', 'computer vision']
    reg = ['regression']
    classifiers = ['classifier', 'forest', 'classification', 'identification']
    ts = ['time series', 'forecasting']
    auto = ['automation', 'automated', 'automating']
    other = ['other', 'unknown', 'big data', 'automation', 'sql', 'visual', 'fuzzy']

    options = [('Natural Language Processing', nlp),('Neural Network', nn),
            ('Machine Vision',mv),('Regression', reg), ('Classification', classifiers),("Large Language Model", llm),
                ('Time Series Forecast', ts),("Automation", auto), ("Unspecified Machine Learning", gml), ('Other', other)]
    

    def sort_by_techniques(technique):
        if pd.isnull(technique):
            return None
        for cat, list in options:
            if technique is not None:
                if any(match in technique.lower() for match in list):
                    return cat
        
        return 'review'
    
     #clean up technique
    nlp2 = ['nlp', 'natural language']
    nn2 = ['neural network', 'lstm']
    gml2 = ['machine learning', 'ml', 'ai', 'artificial intelligence', 
        'reinforcement']
    llm2 = ['large language']
    mv2 = ['machine vision', 'computer vision', "image"]
    reg2 = ['regression']
    classifiers2 = ['classifier', 'forest', 'classification']
    ts2 = ['time series', 'forecasting']
    auto2 = ['automation', 'automated', 'automating']
    other2 = ['big data', 'automation']

    options2 = [('Natural Language Processing', nlp2),('Neural Network', nn2),
            ('Machine Vision',mv2),('Regression', reg2), ('Classification', classifiers2),("Large Language Model", llm2),
                ('Time Series Forecast', ts2),("Automation", auto), ("Unspecified Machine Learning", gml2), ('Other', other2)]
    
    def sort_by(df):
        for cat, list in options2:
            if df['tech_stg2'] is None:
                if any(match in df['Summary'].lower() for match in list):
                    return cat

   
    
    df['tech_stg1'] = df['Techniques'].apply(lambda x: sort_by_techniques(x))

    df['tech_stg2'] = None
    df['tech_stg2'] = df.apply(sort_by, axis=1)

    df['tech_clean'] = df['tech_stg1'].combine_first(df['tech_stg2'])

    

    top5_tech  = df.groupby('tech_clean')['Use_Case_ID'].count().sort_values(ascending=False).head(5).reset_index()['tech_clean'].tolist()
    df['tech_edit'] = df['tech_clean'].apply(lambda x: x if x in top5_tech else 'All Other Techniques')

    df['tech_edit'] = np.where(df['tech_edit'] == 'Other', 'All Other Techniques', df['tech_edit'])


    # clean up department
    top5_dep  = df.groupby('Department')['Use_Case_ID'].count().sort_values(ascending=False).head(5).reset_index()['Department'].tolist()
    df['dep_edit'] = df['Department'].apply(lambda x: x if x in top5_dep else 'All Other Departments')

    return df




def create_chart_xy(df, col, col_count):
    
    dict = {}
    # for now
    
    values = df[col].unique().tolist()
    values.sort()
    # 5 wide means first is at 3 then +5
    i = 0
    for v in values:
        dict[v] = i
        i += 1

    df['rn'] = df.sort_values([col,'Use_Case_ID'], ascending=[True,True]) \
             .groupby([col]) \
             .cumcount()
    df['x'] = (df[col].map(dict) * 7) + (df['rn'] % col_count) +1
    df['y'] = df['rn'] // col_count + 1

    x_name = col + '_x'
    y_name = col + '_y'
    df = df.rename(columns={'x': x_name, 'y': y_name})
    df.drop(columns=['rn'], inplace=True)

    return df

def create_orderd_cats(df, col):
    cats = df.groupby(col)['Use_Case_ID'].count().reset_index().sort_values('Use_Case_ID', ascending=False)[col].to_list()

    # put others first
    if 'Development' in cats:
        cats.remove('No Development Stage Indicated')
        cats.insert(0, 'No Development Stage Indicated')
    elif "All Other Techniques" in cats:
        cats.remove('All Other Techniques')
        cats.insert(0, 'All Other Techniques')
    
    return cats
    
def starting_grid(df, start_col, max_x = 27):

    if start_col == 'dev_edit':
        c = 'dev'

    elif start_col == 'tech_edit':
        c = 't'
    elif start_col == 'dep_edit':
        c = 'dep'
    
    
    col = f"start_{c}"
    x_name = col + '_x'
    y_name = col + '_y'

    #TODO

    ordered_cats = create_orderd_cats(df, start_col)

    df[start_col] = pd.Categorical(df[start_col], ordered=True, 
                              categories=ordered_cats)
        
    df = df.sort_values([start_col, 'Use_Case_ID']).reset_index(drop=True) 


    df['rn'] = df.index

    df[x_name] = df['rn'] % max_x + 1
    df[y_name] = df['rn'] // max_x + 1
 
    return df 

def split_grid(df, split_col, max_x = 27, max_y=27 ):

    if split_col == 'dev_edit':
        c = 'dev'
    elif split_col == 'tech_edit':
        c = 't'
    elif split_col == 'dep_edit':
        c = 'dep'
    
    
    col = f"split_{c}"

    #df = df.sort_values([split_col, 'Use_Case_ID']).reset_index(drop=True) 

    # df['rn'] = df.index 
    x_name = col + '_x'
    y_name = col + '_y'

    df[x_name] = df['rn'] % max_x + 1

    dict = {}
    i = 0
    for val in df[split_col].unique():
        dict[val] = i
        i += 3
        
    df[y_name] = df.apply(lambda x: x['rn'] // max_y + 1 + dict[x[split_col]], axis=1)
  
    return df

There are 710 models
* every model has a unique case id and department code (should use department)
* 
* 10% are missing agency. adding new col for chart to just replace with department name
    * Departments either have agency or dont (HUD, Labor, Treasury, VA, and EPA have no agency info)
* Only Agriculture and Health and Human Services have office info
* 

In [27]:
(data.isnull().sum() / len(data)) * 100

Use_Case_ID           0.000000
Department_Code       0.000000
Agency               10.704225
Office               72.394366
Title                 0.000000
Summary               0.000000
Development_Stage    45.211268
Techniques           55.633803
Source_Code          97.605634
Department            0.000000
dtype: float64

In [41]:
pd.set_option('display.max_colwidth', None)
data[data.duplicated('Summary', keep=False)][["Title", "Summary"]]

# exact duplicates?
#these look to be different use cases, so we'll keep them all
#data[data['Use_Case_ID'].isin(['DOC-0024-2023', 'DOC-0025-2023'])]['Summary'].values

#data[data['Use_Case_ID'].isin(['DOT-0007-2023', 'DOT-0008-2023'])]['Summary'].values

#data[data['Use_Case_ID'].isin(['HHS-0006-2023', 'HHS-0007-2023'])]['Summary'].values

,Title,Summary
140,Fluid migration from well-to-well communication will be inputted in AI to determine a costs-benefit analysis,"This project will develop an ML algorithm to predict the time when a \ngrowing fracture will reach the monitored well. The ML workflow will be \ntrained on the distinctive tensile strain signature that precedes the \ngrowing fracture. The new workflow will be designed to work in \nconjunction with the fracture warning ML workflow developed in EY21. \nTogether, these workflows will: (1) provide an early warning of well-to-\nwell communication, (2) predict the measured depths where the \ncommunication will happen, and (3) provide an estimated time until the \nbeginning of well-to-well communication."
159,To drive insights on the dependencies between the natural gas and electricity sectors to increase reliability of the NG system,Commercially available models will be used to generate predictive \nscenarios
160,"To drive insights on the power system reliability, cost, and operations during the energy transition with and without FECM technologies",Commercially available models will be used to generate predictive \nscenarios
243,Using recursive neural networks and using fiber optic cables to recognize strain patterns and warn operators a fracture is coming.,"This project will develop an ML algorithm to predict the time when a \ngrowing fracture will reach the monitored well. The ML workflow will be \ntrained on the distinctive tensile strain signature that precedes the \ngrowing fracture. The new workflow will be designed to work in \nconjunction with the fracture warning ML workflow developed in EY21. \nTogether, these workflows will: (1) provide an early warning of well-to-\nwell communication, (2) predict the measured depths where the \ncommunication will happen, and (3) provide an estimated time until the \nbeginning of well-to-well communication."
310,Form Recognizer for Benefits Forms,Custom machine learning model to extract data from complex forms to tag data entries to field headers. The input is a document or scanned image of the form and the output is a JSON response with key/value pairs extracted by running the form against the custom trained model.
316,Data Ingestion of Payroll Forms,Custom machine learning model to extract data from complex forms to tag data entries to field headers. The input is a document or scanned image of the form and the output is a JSON response with key/value pairs extracted by running the form against the custom trained model.
392,R+2:18eDIRECT: Clarivate,AI to identify drug repurposing candidates
393,ReDIRECT: AriScience,AI to identify drug repurposing candidates
696,Machine learning models to predict disease progression among veterans with hepatitis C virus,A machine learning model is used to predict disease progression among veterans with hepatitis C virus.
701,Reinforcement learning evaluation of treatment policies for patients with hepatitis C virus,A machine learning model is used to predict disease progression among veterans with hepatitis C virus.


In [4]:


#which department has the most missing agency data
(data.groupby('Department')['Agency'].apply(lambda x: (x.isnull().sum() / len(x)) * 100))
#(data.groupby('Department')['Office'].apply(lambda x: (x.isnull().sum() / len(x)) * 100))


Department
Department of Agriculture                          0.0
Department of Commerce                             0.0
Department of Education                            0.0
Department of Energy                               0.0
Department of Health and Human Services            0.0
Department of Homeland Security                    0.0
Department of Housing and Urban Development      100.0
Department of Interior                             0.0
Department of Justice                              0.0
Department of Labor                              100.0
Department of State                                0.0
Department of Transportation                       0.0
Department of Treasury                           100.0
Department of Veterans Affairs                   100.0
National Aeronautics and Space Administration      0.0
National Archives and Records Administration       0.0
Social Security Administration                     0.0
U.S. Agency for International Development          0.0

In [27]:
df

,Use_Case_ID,Department_Code,Agency,Office,Title,Summary,Development_Stage,Techniques,Source_Code,Department,...,split_dev_x,split_dev_y,start_t_x,start_t_y,split_t_x,split_t_y,start_dep_x,start_dep_y,split_dep_x,split_dep_y
0,DOI-0000-2023,DOI,BLM,NaN,Land Use Plan Document and Data Mining and Ana...,"Exploring the potential to identify patterns, ...",Planned (not in production),Natural Language Processing and Geo Classifica...,NaN,Department of Interior,...,19,33,24,18,24,24,1,1,1,1
1,DOI-0001-2023,DOI,BOR,NaN,Seasonal/Temporary Wetland/Floodplain Delineat...,Reclamation was interested in determining if r...,Completed,Image classification using Joint Unsupervised ...,NaN,Department of Interior,...,17,16,13,4,13,4,2,1,2,1
2,DOI-0002-2023,DOI,BOR,NaN,Data Driven Sub-Seasonal Forecasting of Temper...,"Reclamation has run 2, year-long prize competi...",Development (not in production),"Range of data driven, AI/ML techniques (e.g. r...",NaN,Department of Interior,...,5,27,14,4,14,4,3,1,3,1
3,DOI-0003-2023,DOI,BOR,NaN,Data Driven Streamflow Forecasting,"Reclamation, along with partners from the CEAT...",Development (not in production),"Range of data driven, AI/ML techniques (e.g. L...",NaN,Department of Interior,...,6,27,21,23,21,32,4,1,4,1
4,DOI-0004-2023,DOI,BOR,NaN,Snowcast Showdown,Reclamation partnered with Bonneville Power Ad...,Development and Acquisition,"Range of data driven, AI/ML techniques",NaN,Department of Interior,...,7,27,15,11,15,14,5,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,VA-0035-2023,VA,NaN,NaN,Extraction of family medical history from pati...,This pilot project uses TIU documentation on A...,NaN,NaN,NaN,Department of Veterans Affairs,...,20,12,14,8,14,8,4,27,4,42
706,VA-0036-2023,VA,NaN,NaN,VA /IRB approved research study for finding co...,This IRB approved research study uses a rando...,NaN,NaN,NaN,Department of Veterans Affairs,...,21,12,15,8,15,8,5,27,5,42
707,VA-0037-2023,VA,NaN,NaN,Interpretation/triage of eye images,Artificial intelligence supports triage of eye...,NaN,NaN,NaN,Department of Veterans Affairs,...,22,12,8,27,8,39,6,27,6,42
708,VA-0038-2023,VA,NaN,NaN,Screening for esophageal adenocarcinoma,National VHA administrative data is used to ad...,NaN,NaN,NaN,Department of Veterans Affairs,...,23,12,16,8,16,8,7,27,7,42


In [28]:
(df.groupby('Department')['Development_Stage'].apply(lambda x: (x.isnull().sum() / len(x)) * 100))


Department
Department of Agriculture                          0.000000
Department of Commerce                           100.000000
Department of Education                          100.000000
Department of Energy                              99.438202
Department of Health and Human Services            0.000000
Department of Homeland Security                    0.000000
Department of Housing and Urban Development      100.000000
Department of Interior                             0.000000
Department of Justice                              0.000000
Department of Labor                                0.000000
Department of State                              100.000000
Department of Transportation                      35.714286
Department of Treasury                             0.000000
Department of Veterans Affairs                   100.000000
National Aeronautics and Space Administration      0.000000
National Archives and Records Administration       0.000000
Social Security Administratio

In [26]:
data[data["Department"] == 'Department of Health and Human Services'].groupby("Office", dropna=False).size()

Office
ACF Children's Bureau                                                                            1
AHRQ                                                                                             2
BARDA (CBRN & DRIVe)                                                                             2
BARDA (CBRN)                                                                                     4
BARDA (DRIVe)                                                                                    5
CBER/OBPV/DABRA                                                                                  3
CDER/Office of Generic Drugs                                                                     4
CDER/Office of New Drugs                                                                         1
CDER/Office of Pharmaceutical Quality (OPQ)                                                      1
CDER/Office of Strategic Programs (OSP)                                                          1
CDE

In [13]:
for col in data.columns:
    print(data.groupby(col, dropna=False).size().sort_values(ascending=False, na_position='first').head())

Use_Case_ID
DHS-0000-2023    1
HHS-0055-2023    1
HHS-0079-2023    1
HHS-0080-2023    1
HHS-0081-2023    1
dtype: int64
Department_Code
DOE    178
HHS    157
DOC     49
DHS     41
VA      40
dtype: int64
Agency
National Energy Technology Laboratory    127
NaN                                       76
NIH                                       47
FDA                                       44
USDA                                      39
dtype: int64
Office
NaN                                                   514
Centers for Medicare & Medicaid Services (CMS)         23
National Institutes of Health (NIH) NLM                14
NCTR                                                   12
CDER/Office of Surveillance and Epidemiology (OSE)     11
dtype: int64
Title
Machine Learning for Occupant Safety Research    2
Burn & Blast MCMs: Philips                       2
First Guess Excessive Rainfall Outlook           2
PyForecast                                       1
Privileged Material Identificat